<a href="https://colab.research.google.com/github/demircanserdar/CS412/blob/main/textpreprocessing/TextPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import regex
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np

nltk.download('stopwords')
nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')

def filter_text(text, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(text.lower())
    filtered_text = [regex.sub('[^a-z ]+', '', w) for w in word_tokens]
    filtered_text = [regex.sub('[ ][ ]+', '', w) for w in filtered_text]
    filtered_text = [regex.sub('[0-9]', '', w) for w in filtered_text]
    filtered_text = [wordnet_lemmatizer.lemmatize(w, 'v') for w in filtered_text if not w in stop_words and len(w) > 2 and len(w) < 50]
    return " ".join(filtered_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
df = pd.read_csv("bugs-train.csv")
df["filtered_text"] = df["summary"].apply(lambda x : filter_text(x, stop))
df.head()

,bug id,summary,severity,filtered_text
0,365569,Remove workaround from bug 297227,normal,remove workaround bug
1,365578,Print Preview crashes on any URL in gtk2 builds,critical,print preview crash url gtk build
2,365582,Lines are not showing in table,major,line show table
3,365584,Firefox render ÛÏsimplified ArabicÛ font fa...,normal,firefox render simplify arabic font face incor...
4,365597,Crash [@ nsINodeInfo::NodeInfoManager],critical,crash nsinodeinfo nodeinfomanager


In [4]:
df['classification'] = df['severity'].replace(['enhancement', 'minor', 'normal', 'major', 'blocker','critical', 'trivial'],[1,2,3,4,5,6,0])
df['classification'] = df['classification'].dropna()
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()
df['classification'].astype(int)

0         3
1         6
2         4
3         3
4         6
         ..
159993    3
159994    3
159995    6
159996    6
159997    3
Name: classification, Length: 159998, dtype: int64

In [5]:
df.head()

,bug id,summary,severity,filtered_text,classification
0,365569,Remove workaround from bug 297227,normal,remove workaround bug,3
1,365578,Print Preview crashes on any URL in gtk2 builds,critical,print preview crash url gtk build,6
2,365582,Lines are not showing in table,major,line show table,4
3,365584,Firefox render ÛÏsimplified ArabicÛ font fa...,normal,firefox render simplify arabic font face incor...,3
4,365597,Crash [@ nsINodeInfo::NodeInfoManager],critical,crash nsinodeinfo nodeinfomanager,6


In [6]:
df.to_csv("preprocessed_text.csv")

In [7]:
df = pd.read_csv("bugs-test.csv")
df["filtered_text"] = df["summary"].apply(lambda x : filter_text(x, stop))
df.to_csv("preprocessed_submission.csv")